In [ ]:
#!pip install skorch

In [ ]:
import numpy as np
import random
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable  
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset,DataLoader

from pathlib import Path
import matplotlib.pyplot as plt

import sklearn


from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

In [ ]:
def load(paths,initial='clients'):
    
    data = list()
    labels = list()
    
    data_folder = Path("./Data/")
    
    client_names = ['{}_{}'.format(initial, i+1) for i in range(len(paths))]
    
    total_clients={client_names[i] : [] for i in range(len(client_names))}
    
    # loop over the input
    for j in range(len(paths)):
        
        # load the data and extract the class labels
        target = pd.read_csv(data_folder / paths[j], usecols=[2, 5],
                             index_col=0, header=0)
        target=target.loc["20200601":"20200831"][:]

        
        target=target.to_numpy()

        #sc = preprocessing.MinMaxScaler()
        #target = sc.fit_transform(target)
        #sc = preprocessing.StandardScaler().fit(target)
        #target = sc.transform(target)
        
        ##target=target.values.tolist()
        
        #total_clients[client_names[j]]=list(zip((Data, target)))
        
        total_clients[client_names[j]]=target
        
        #{client_names[i] : list(zip((Data, target)))}
        
        
    return  total_clients

In [ ]:
#data_paths=['building_1_.csv','building_2_.csv','building_3_.csv','building_4_.csv','building_5_.csv','building_6_.csv','building_7_.csv',
 #           'building_9_.csv','building_10_.csv','building_11_.csv','building_12_.csv','building_13_.csv','building_14_.csv','building_16_.csv',
 #           'building_17_.csv','building_18_.csv','building_19_.csv','building_20_.csv','building_21_.csv','building_22_.csv','building_23_.csv','building_24_.csv',
 #           'building_25_.csv','building_26_.csv','building_27_.csv','building_28_.csv','building_29_.csv','building_30_.csv','building_31_.csv','building_32_.csv',
 #           'building_33_.csv','building_34_.csv','building_35_.csv','building_36_.csv','building_37_.csv','building_38_.csv','building_39_.csv','building_40_.csv',
 #           'building_41_.csv','building_42_.csv','building_43_.csv','building_44_.csv','building_45_.csv','building_46_.csv','building_47_.csv','building_48_.csv',
 #           'building_49_.csv','building_50_.csv','building_51_.csv','building_53_.csv','building_54_.csv','building_56_.csv',
  #          'building_57_.csv','building_59_.csv','building_60_.csv','building_61_.csv','building_62_.csv','building_63_.csv','building_64_.csv',
  #          'building_65_.csv','building_66_.csv','building_67_.csv','building_68_.csv','building_69_.csv','building_70_.csv','building_71_.csv','building_72_.csv',
  #          'building_73_.csv','building_74_.csv','building_75_.csv','building_76_.csv','building_77_.csv','building_78_.csv','building_79_.csv','building_80_.csv','building_81_.csv']

data_paths=['building_1_.csv','building_2_.csv','building_3_.csv','building_4_.csv','building_5_.csv','building_6_.csv','building_7_.csv',
            'building_9_.csv','building_10_.csv','building_11_.csv','building_12_.csv','building_13_.csv','building_14_.csv','building_16_.csv',
            'building_17_.csv','building_18_.csv','building_19_.csv','building_20_.csv','building_21_.csv','building_22_.csv']


#data_paths=['building_23_.csv','building_24_.csv',
 #           'building_25_.csv','building_26_.csv','building_27_.csv','building_28_.csv','building_29_.csv','building_30_.csv','building_31_.csv','building_32_.csv',
 #           'building_33_.csv','building_34_.csv','building_35_.csv','building_36_.csv','building_37_.csv','building_38_.csv','building_39_.csv','building_40_.csv',
  #          'building_41_.csv']


#data_paths=['building_42_.csv','building_43_.csv','building_44_.csv','building_45_.csv','building_46_.csv','building_47_.csv','building_48_.csv',
#            'building_49_.csv','building_50_.csv','building_51_.csv','building_53_.csv','building_54_.csv','building_56_.csv',
#            'building_57_.csv','building_59_.csv','building_60_.csv','building_61_.csv','building_62_.csv','building_63_.csv','building_64_.csv']


#data_paths=['building_65_.csv','building_66_.csv','building_67_.csv','building_68_.csv','building_69_.csv','building_70_.csv','building_71_.csv','building_72_.csv',
    #        'building_73_.csv','building_74_.csv','building_75_.csv','building_76_.csv','building_77_.csv','building_78_.csv','building_79_.csv','building_80_.csv','building_81_.csv']

#data_paths=['building_1_.csv']
clients=load(data_paths,initial='clients')


In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []
  

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]

        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)



#process and batch the test data for each client

client_names = ['{}_{}'.format('clients', i+1) for i in range(len(data_paths))]

x=dict()
y=dict()

seq_length = 24

for i in range(len(data_paths)):
    
    x[client_names[i]], y[client_names[i]] = sliding_windows(clients[client_names[i]], seq_length)
    


In [ ]:
train_size = int(len(y[client_names[i]]) * 0.75)
test_size = len(y[client_names[i]]) - train_size

dataX = dict()
dataY= dict()

trainX= dict()
trainY= dict()

testX= dict()
testY= dict()

for i in range(len(data_paths)):
    
    dataX[client_names[i]] = Variable(torch.Tensor(np.array(x[client_names[i]]))).to(device)
    dataY[client_names[i]] = Variable(torch.Tensor(np.array(y[client_names[i]]))).to(device)

    trainX[client_names[i]] = Variable(torch.Tensor(np.array(x[client_names[i]][0:train_size]))).to(device)
    trainY[client_names[i]] = Variable(torch.Tensor(np.array(y[client_names[i]][0:train_size]))).to(device)

    testX[client_names[i]] = Variable(torch.Tensor(np.array(x[client_names[i]][train_size:len(x[client_names[i]])]))).to(device)
    testY[client_names[i]] = Variable(torch.Tensor(np.array(y[client_names[i]][train_size:len(y[client_names[i]])]))).to(device)


In [ ]:
class Net(nn.Module):

    def __init__(self, l1, l2):
        super(Net, self).__init__()
        
        self.num_classes = 1
        self.num_layers = 1
        self.input_size = 1
        self.l1 = l1
        self.l2 = l2
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=1, hidden_size=l1,
                            num_layers=1, batch_first=True)
        
        self.fc1 = nn.Linear(l1, l2)
        self.fc2 = nn.Linear(l2, 1)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.l1)).to(device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.l1)).to(device)
        
        # Propagate input through LSTM
        out, _ = self.lstm(x, (h_0, c_0))
        out = out[:, -1, :]
        #h_out = h_out.view(-1, self.hidden_size1)
        
        out = self.fc1(out)
        out = self.fc2(out)
        #out = torch.clip(out, 0)
        
        return out

In [ ]:
All_parameters=dict()

All_results=dict()

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create the sklearn model for the network

model_init_batch_epoch_CV = NeuralNetRegressor(Net, verbose=0, train_split=False, lr=0.01, batch_size=24, optimizer=torch.optim.Adam, criterion=nn.MSELoss(), device=device)



#epochs = np.random.randint(80, 300, 20)
#epochs=epochs.tolist()
epochs=[103, 148, 190, 247, 291]

#hidden_size1=np.random.randint(24, 70, 20)
#hidden_size1=hidden_size1.tolist()
hidden_size1=[24, 32, 44, 56, 68]

hidden_size2=[85, 105, 127, 148, 198]
#second_layer_neurons=np.random.randint(80, 200, 20)
#second_layer_neurons=second_layer_neurons.tolist()

param_grid = {
	'max_epochs':epochs,
	'module__l1': hidden_size1,
  'module__l2': hidden_size2,
  'lr': [0.01],
  'optimizer': [torch.optim.Adam],
  'criterion': [nn.MSELoss],
  'batch_size':[24]
	}


grid = GridSearchCV(model_init_batch_epoch_CV, 
                    param_grid, refit=False,
                    cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)




#neg_mean_absolute_percentage_error

client_names = ['{}_{}'.format('clients', i+1) for i in range(len(data_paths))]



for client in client_names:


    grid_result = grid.fit(trainX[client].cpu(), trainY[client].cpu())
    
    
    All_parameters[client]=grid_result.best_params_
    
    All_results[client]=grid_result.cv_results_


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 50.6min
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed: 136.6min finished


In [ ]:
All_parameters=pd.DataFrame(All_parameters)
All_parameters.to_csv("output.csv")

In [ ]:
All_results=pd.DataFrame(All_results)
All_results.to_csv("results.csv")